In [ ]:
# !pip install kaggle
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !kaggle competitions download -c competitive-data-science-predict-future-sales
# !pip install catboost
# !pip install category-encoders
# !pip install xgboost
# !pip install lightgbm
# !sudo apt-get install unzip
# !unzip \*.zip

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import xgboost as xgb

In [ ]:
import lightgbm as lgbm

In [ ]:
import gc

In [ ]:
from itertools import product

In [ ]:
from catboost import CatBoostRegressor,cv

In [ ]:
import category_encoders as ce

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import re

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
sales = pd.read_csv("sales_train.csv")

In [ ]:
sales.date=pd.to_datetime(sales['date'],format = '%d.%m.%Y')

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
sales = sales[sales.item_price < 100000]
sales = sales[sales.item_cnt_day < 1001]

median = sales[(sales.shop_id == 32) & (sales.item_id == 2973) & (sales.date_block_num == 4) & (
            sales.item_price > 0)].item_price.median()
sales.loc[sales.item_price < 0, 'item_price'] = median

In [ ]:
sales.loc[sales.shop_id == 0, 'shop_id'] = 57
sales.loc[sales.shop_id == 1, 'shop_id'] = 58
sales.loc[sales.shop_id == 10, 'shop_id'] = 11

test.loc[test.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 10, 'shop_id'] = 11


In [ ]:
g = sales.groupby([pd.Grouper("date_block_num"),pd.Grouper("shop_id"),pd.Grouper("item_id")]).item_cnt_day.agg(sum).reset_index()

In [ ]:
p = sales.groupby([ pd.Grouper("date_block_num"),pd.Grouper("shop_id"),pd.Grouper("item_id")]).item_price.agg("median").reset_index()

In [ ]:
unique_shop_id = set( g.shop_id.unique().tolist() + test.shop_id.unique().tolist() )
unique_item_id = set( g.item_id.unique().tolist() + test.item_id.unique().tolist() )
unique_date_block = 33

In [ ]:
items_not_train = set(test.item_id.unique()) - set(sales.item_id.unique())

In [ ]:
len(items_not_train)

363

In [ ]:

dfs = []

for i in range(0,34):

  unq_shops = g[g.date_block_num==i].shop_id.unique()
  unq_items = np.unique( ( g[g.date_block_num==i].item_id.unique().tolist() ) )

  dfs.append( list( product([i],unq_shops,unq_items ) ) )

fd = pd.DataFrame( np.vstack(dfs) )



In [ ]:
fd = fd.rename( columns = { 0:"date_block_num", 1:"shop_id", 2:"item_id" } )

In [ ]:
fd[["shop_id","item_id"]].drop_duplicates()

,shop_id,item_id
0,2,19
1,2,27
2,2,28
3,2,29
4,2,32
...,...,...
10913761,59,21973
10913769,59,22004
10913770,59,22005
10913771,59,22006


In [ ]:
fd

,date_block_num,shop_id,item_id
0,0,2,19
1,0,2,27
2,0,2,28
3,0,2,29
4,0,2,32
...,...,...,...
10913799,33,59,22162
10913800,33,59,22163
10913801,33,59,22164
10913802,33,59,22166


In [ ]:
sales.item_id.nunique()

21806

In [ ]:
g_ = pd.merge( fd,g,how="left",on=["date_block_num","shop_id","item_id"])

In [ ]:
g_ = pd.merge( g_,p,how="left",on=["date_block_num","shop_id","item_id"])

In [ ]:
del fd
del g
del dfs 

In [ ]:
gc.collect()

150

In [ ]:
g_.fillna(0,inplace=True)

In [ ]:
g_

,date_block_num,shop_id,item_id,item_cnt_day,item_price
0,0,2,19,0.0,0.0
1,0,2,27,1.0,2499.0
2,0,2,28,0.0,0.0
3,0,2,29,0.0,0.0
4,0,2,32,0.0,0.0
...,...,...,...,...,...
10913799,33,59,22162,0.0,0.0
10913800,33,59,22163,0.0,0.0
10913801,33,59,22164,0.0,0.0
10913802,33,59,22166,0.0,0.0


In [ ]:
g_.item_cnt_day = g_.item_cnt_day.clip(0,20)

In [ ]:
g_.isna().sum()

date_block_num    0
shop_id           0
item_id           0
item_cnt_day      0
item_price        0
dtype: int64

In [ ]:
g_

,date_block_num,shop_id,item_id,item_cnt_day,item_price
0,0,2,19,0.0,0.0
1,0,2,27,1.0,2499.0
2,0,2,28,0.0,0.0
3,0,2,29,0.0,0.0
4,0,2,32,0.0,0.0
...,...,...,...,...,...
10913799,33,59,22162,0.0,0.0
10913800,33,59,22163,0.0,0.0
10913801,33,59,22164,0.0,0.0
10913802,33,59,22166,0.0,0.0


In [ ]:
test["date_block_num"] = 34

In [ ]:
test

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34
...,...,...,...,...
214195,214195,45,18454,34
214196,214196,45,16188,34
214197,214197,45,15757,34
214198,214198,45,19648,34


In [ ]:
g_ = pd.concat([g_,test],axis=0)

In [ ]:
g_.drop("ID",axis=1,inplace=True)

In [ ]:
g_

,date_block_num,shop_id,item_id,item_cnt_day,item_price
0,0,2,19,0.0,0.0
1,0,2,27,1.0,2499.0
2,0,2,28,0.0,0.0
3,0,2,29,0.0,0.0
4,0,2,32,0.0,0.0
...,...,...,...,...,...
214195,34,45,18454,NaN,NaN
214196,34,45,16188,NaN,NaN
214197,34,45,15757,NaN,NaN
214198,34,45,19648,NaN,NaN


In [ ]:
g_.item_cnt_day.fillna(0,inplace=True)
g_.item_price.fillna(0,inplace=True)

In [ ]:
g_['month'] = (g_.date_block_num % 12) + 1

In [ ]:
g_

,date_block_num,shop_id,item_id,item_cnt_day,item_price,month
0,0,2,19,0.0,0.0,1
1,0,2,27,1.0,2499.0,1
2,0,2,28,0.0,0.0,1
3,0,2,29,0.0,0.0,1
4,0,2,32,0.0,0.0,1
...,...,...,...,...,...,...
214195,34,45,18454,0.0,0.0,11
214196,34,45,16188,0.0,0.0,11
214197,34,45,15757,0.0,0.0,11
214198,34,45,19648,0.0,0.0,11


In [ ]:
items = pd.read_csv("items.csv")
shops = pd.read_csv("shops.csv")
categories = pd.read_csv('item_categories.csv')

In [ ]:
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [ ]:
def name_correction(x):
    x = x.lower() # all letters lower case
    x = x.partition('[')[0] # partition by square brackets
    x = x.partition('(')[0] # partition by curly brackets
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) # remove special characters
    x = x.replace('  ', ' ') # replace double spaces with single spaces
    x = x.strip() # remove leading and trailing white space
    return x

In [ ]:
#Shop treating
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops = shops[['shop_id','city']]


#Categories treating
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x: x[0].strip())
# if subtype is nan then type
categories['subtype'] = categories['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
categories = categories[['item_category_id','type', 'subtype']]


#Items treating
# split item names by first bracket
items["name1"], items["name2"] = items.item_name.str.split("[", 1).str
items["name1"], items["name3"] = items.item_name.str.split("(", 1).str

# replace special characters and turn to lower case
items["name2"] = items.name2.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
items["name3"] = items.name3.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

# fill nulls with '0'
items = items.fillna('0')

items["item_name"] = items["item_name"].apply(lambda x: name_correction(x))

# return all characters except the last if name 2 is not "0" - the closing bracket
items.name2 = items.name2.apply( lambda x: x[:-1] if x !="0" else "0")

items["type"] = items.name2.apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0] )
items.loc[(items.type == "x360") | (items.type == "xbox360") | (items.type == "xbox 360") ,"type"] = "xbox 360"
items.loc[ items.type == "", "type"] = "mac"
items.type = items.type.apply( lambda x: x.replace(" ", "") )
items.loc[ (items.type == 'pc' )| (items.type == 'pс') | (items.type == "pc"), "type" ] = "pc"
items.loc[ items.type == 'рs3' , "type"] = "ps3"

group_sum = items.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()
drop_cols = []
for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] <40:
        drop_cols.append(cat)
items.name2 = items.name2.apply( lambda x: "other" if (x in drop_cols) else x )
items = items.drop(["type"], axis = 1)

items.name2 = LabelEncoder().fit_transform(items.name2)
items.name3 = LabelEncoder().fit_transform(items.name3)

items.drop(["item_name", "name1"],axis = 1, inplace= True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


In [ ]:
g_ = pd.merge(g_, items, on = 'item_id', how='left')
g_ = pd.merge(g_, shops, on = 'shop_id', how='left')
g_ = pd.merge(g_, categories, on = 'item_category_id', how='left')


In [ ]:
days = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
g_['days'] = g_['month'].map(days).astype(np.int8)

#Label encoding
g_['city'] = LabelEncoder().fit_transform(g_['city'])
g_['subtype'] = LabelEncoder().fit_transform(g_['subtype'])
g_['type'] = LabelEncoder().fit_transform(g_['type'])


In [ ]:
g_.item_category_id.isna().sum()

0

In [ ]:
ig = set (g_.item_id.unique())
it = set (items.item_id.unique())
print( len (ig-it) )

0


In [ ]:
g_.fillna(0,inplace=True)
g_ = g_.sort_values(by="date_block_num")

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
g_ = reduce_mem_usage(g_)

Memory usage of dataframe is 1114.31 MB
Memory usage after optimization is: 275.92 MB
Decreased by 75.2%


In [ ]:
def generate_lag_f(df,lag,col,tg_name="lag"):

  tmp = df[["date_block_num","item_id","shop_id",col]].copy()
  tmp.columns = ["date_block_num","item_id","shop_id",f"{tg_name}_{lag}"]
  tmp.date_block_num += lag

  return tmp

In [ ]:
def generate_groupby_features(df,col_names,target_col,new_target_name,f,how="left",lag=1):
    new_target_name = f"{new_target_name}_lag_{lag}"
    x = df.groupby(col_names)[target_col].agg(f).reset_index().rename(columns={target_col:new_target_name})
    x.date_block_num += lag
    df = pd.merge(left=df,right=x,how=how,on=col_names)
    df[new_target_name].fillna(0,inplace=True)
    print(new_target_name,"created.")
    
    return df

In [ ]:
def generate_weekly_sales(cols,f,lag):
    dfs = []
    
    
    for i in sales_.date_block_num.unique():
        k = sales_[sales_.date_block_num == i].copy()
        l = k.groupby(cols).item_cnt_day.agg(f).reset_index()
        l["week_num"] = l.date.dt.isocalendar().week
        x = pd.crosstab(columns=l["week_num"],index=[ l[c] for c in cols[1:] ],values=l["item_cnt_day"],aggfunc=sum).reset_index().fillna(0).rename_axis(None,axis=1)
        
        if ( l.week_num.nunique() == 6 ):
            wc = np.sort(np.unique(l.week_num))[1:].tolist()
            
        else :
            wc = np.unique(l.week_num).tolist()
                        
        cols_to_sel = cols[1:] + wc
        
        x = x[cols_to_sel]
        x.columns = cols[1:] + list(range(len(wc)))
        x["date_block_num"] = i    
        dfs.append(x)
       
    weekly_sales = pd.DataFrame(np.vstack(dfs),columns=dfs[0].columns)
    weekly_sales.drop([0,1,2],axis=1,inplace=True)
    weekly_sales.date_block_num += lag
    weekly_sales = weekly_sales.rename(columns={
        0:"_".join ( (*(cols[1:]),f"week_0_{f}_lag_{lag}")),
        1:"_".join ( (*(cols[1:]),f"week_1_{f}_lag_{lag}")),
        2:"_".join ( (*(cols[1:]),f"week_2_{f}_lag_{lag}")),
        3:"_".join ( (*(cols[1:]),f"week_3_{f}_lag_{lag}")),
        4:"_".join ( (*(cols[1:]),f"week_4_{f}_lag_{lag}"))
    })
    print(f"weekly data {f} created.")
    return weekly_sales

In [ ]:
def generate_count_features(df,col_name,target_col="item_cnt_day",lag=1):
    enc_col = "_".join([*(col_name[1:]),f"counts_lag_{lag}"])
    print(enc_col,"created.")
    x = df.groupby(col_name)[target_col].count().reset_index().rename(columns={target_col:enc_col})
    x.date_block_num += lag
    return x

In [ ]:
def generate_mean_encodings(df,col_name,enc_type="mean",fill_val=0,target_col="item_cnt_day"):

  enc_col = '_'.join([f"enc_{enc_type}",*col_name,target_col])

  df[enc_col] = np.nan


  for i in (df.date_block_num.unique()):

    if not i : continue

    tr_idx = (df.date_block_num < i)
    val_idx = (df.date_block_num == i)


    tr, va = df.loc[tr_idx],df.loc[val_idx]
    means = tr.groupby(col_name)[target_col].agg(enc_type).reset_index().copy()
    enc = pd.merge(va[col_name],means,how="left",on=col_name)[[target_col]].set_index(df.loc[val_idx].index)
    df.loc[val_idx,enc_col] = enc[target_col]
  
  df[enc_col].fillna(fill_val,inplace=True)
  print(enc_col,"created.")

In [ ]:
sales_ = pd.merge(left=sales,right=items[['item_id',"item_category_id"]],how="left",on=["item_id"])


In [ ]:
g_ = g_.merge(generate_lag_f(g_,1,"item_cnt_day"),how="left",on=["date_block_num","item_id","shop_id"])
g_ = g_.merge(generate_lag_f(g_,2,"item_cnt_day"),how="left",on=["date_block_num","item_id","shop_id"])
g_ = g_.merge(generate_lag_f(g_,3,"item_cnt_day"),how="left",on=["date_block_num","item_id","shop_id"])


g_ = g_.merge(generate_lag_f(g_,1,"item_price","price_lag"),how="left",on=["date_block_num","item_id","shop_id"])

In [ ]:
sales_

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,2013-01-02,0,59,22154,999.00,1.0,37
1,2013-01-03,0,25,2552,899.00,1.0,58
2,2013-01-05,0,25,2552,899.00,-1.0,58
3,2013-01-06,0,25,2554,1709.05,1.0,58
4,2013-01-15,0,25,2555,1099.00,1.0,56
...,...,...,...,...,...,...,...
2935842,2015-10-10,33,25,7409,299.00,1.0,55
2935843,2015-10-09,33,25,7460,299.00,1.0,55
2935844,2015-10-14,33,25,7459,349.00,1.0,55
2935845,2015-10-22,33,25,7440,299.00,1.0,57


In [ ]:
x = sales_.groupby([pd.Grouper(key="date",freq="14D"),"date_block_num","item_id"]).item_cnt_day.agg('sum').reset_index()

In [ ]:
def get_last_15_day_sale(df,cols,trg,f,lag):
    
    dfs = []
    
    for i in df.date_block_num.unique():
        x = df[df.date_block_num == i].copy()
        
        grp = x.groupby(cols)[trg].agg(f).reset_index().fillna(0).rename_axis(None,axis=1)
        grp["week_num"] = grp.date.dt.week
        x = pd.crosstab(columns=grp["week_num"],index=[ grp[c] for c in cols[1:] ],values=grp[trg],aggfunc=sum).reset_index().fillna(0).rename_axis(None,axis=1)
        
        
        if len (grp.week_num.unique()) == 3:
            wc = np.sort(np.unique(grp.week_num))[1:].tolist()
        else:
            wc = np.unique(grp.week_num).tolist()
        
        cols_to_sel = cols[1:]+wc
        
        x = x[cols_to_sel]
        x.columns = cols[1:] + list(range(len(wc)))
        x["date_block_num"] = i    
        dfs.append(x)
    
    weekly_sales = pd.DataFrame(np.vstack(dfs),columns=dfs[0].columns)
    weekly_sales.date_block_num += 1
    weekly_sales = weekly_sales.rename(columns={
        0:"_".join ( (*(cols[1:]),f"15D_0_{f}_lag_{lag}")),
        1:"_".join ( (*(cols[1:]),f"15D_1_{f}_lag_{lag}")),
        2:"_".join ( (*(cols[1:]),f"15D_2_{f}_lag_{lag}")),
    })
    print("15 day var created")
    return weekly_sales
    
    
    
        
        
        
        
        
        

In [ ]:


for i in range(1,2):
    
    for f in ["sum","mean"]:
        g_ = generate_groupby_features(g_,["date_block_num","item_id"],"item_cnt_day",f"item_{f}_sale",f,lag=i)
        g_ = generate_groupby_features(g_,["date_block_num","shop_id"],"item_cnt_day",f"shop_{f}_sale",f,lag=i)
        g_ = generate_groupby_features(g_,["date_block_num","item_category_id"],"item_cnt_day",f"item_cat_{f}_sale",f,lag=i)
    
    
    

item_sum_sale_lag_1 created.
shop_sum_sale_lag_1 created.
item_cat_sum_sale_lag_1 created.
item_mean_sale_lag_1 created.
shop_mean_sale_lag_1 created.
item_cat_mean_sale_lag_1 created.


In [ ]:


for i in range(1,2):
    
    for f in ["sum","mean"]:
        g_ = generate_groupby_features(g_,["date_block_num","item_id","shop_id"],"item_cnt_day",f"item_shop_{f}_sale",f,lag=i)
        g_ = generate_groupby_features(g_,["date_block_num","item_category_id","shop_id"],"item_cnt_day",f"item_cat_shop_{f}_sale",f,lag=i)
    
    
    
    
    

item_shop_sum_sale_lag_1 created.
item_cat_shop_sum_sale_lag_1 created.
item_shop_mean_sale_lag_1 created.
item_cat_shop_mean_sale_lag_1 created.


In [ ]:
f = "mean"

for i in range(1,2):
    g_ = generate_groupby_features(g_,["date_block_num","item_id"],"item_price",f"item_{f}_price",f,lag=i)
    g_ = generate_groupby_features(g_,["date_block_num","item_category_id"],"item_price",f"item_category_{f}_price",f,lag=i)
    

item_mean_price_lag_1 created.
item_category_mean_price_lag_1 created.


In [ ]:
f = "mean"

for i in range(1,2):
    g_ = generate_groupby_features(g_,["date_block_num","item_id","shop_id"],"item_price",f"item_shop_{f}_price",f,lag=i)
    g_ = generate_groupby_features(g_,["date_block_num","item_category_id","shop_id"],"item_price",f"item_category_shop_{f}_price",f,lag=i)
    

item_shop_mean_price_lag_1 created.
item_category_shop_mean_price_lag_1 created.


In [ ]:
for i in range(1,2):
        g_ = pd.merge(g_,  generate_count_features(sales_,["date_block_num","item_id"],lag=i)  ,how="left",on=["date_block_num","item_id"])
        g_ = pd.merge(g_,  generate_count_features(sales_,["date_block_num","shop_id"],lag=i)  ,how="left",on=["date_block_num","shop_id"])
        g_ = pd.merge(g_, generate_count_features(sales_,["date_block_num","item_category_id"],lag=i)   ,how="left",on=["date_block_num","item_category_id"])
     

item_id_counts_lag_1 created.
shop_id_counts_lag_1 created.
item_category_id_counts_lag_1 created.


In [ ]:
g_.fillna(0,inplace=True)

In [ ]:
col2 = [pd.Grouper(key="date",freq="W"),"shop_id"]
col3 = [pd.Grouper(key="date",freq="W"),"item_id"]
col4 = [pd.Grouper(key="date",freq="W"),"item_category_id"]


In [ ]:
for i in range(1,2):
    for f in ["sum","mean","count"]:
        g_ = pd.merge(g_, generate_weekly_sales(col2,f,lag=i), how="left",on=col2[1:]+["date_block_num"] )
        g_ = pd.merge(g_, generate_weekly_sales(col3,f,lag=i), how="left",on=col3[1:]+["date_block_num"] )
        g_ = pd.merge(g_, generate_weekly_sales(col4,f,lag=i), how="left",on=col4[1:]+["date_block_num"] )
        
    
    

weekly data sum created.
weekly data sum created.
weekly data sum created.
weekly data mean created.
weekly data mean created.
weekly data mean created.
weekly data count created.
weekly data count created.
weekly data count created.


In [ ]:
g_

,date_block_num,shop_id,item_id,item_cnt_day,item_price,month,item_category_id,name2,name3,city,type,subtype,days,lag_1,lag_2,lag_3,price_lag_1,item_sum_sale_lag_1,shop_sum_sale_lag_1,item_cat_sum_sale_lag_1,item_mean_sale_lag_1,shop_mean_sale_lag_1,item_cat_mean_sale_lag_1,item_shop_sum_sale_lag_1,item_cat_shop_sum_sale_lag_1,item_shop_mean_sale_lag_1,item_cat_shop_mean_sale_lag_1,item_mean_price_lag_1,item_category_mean_price_lag_1,item_shop_mean_price_lag_1,item_category_shop_mean_price_lag_1,item_id_counts_lag_1,shop_id_counts_lag_1,item_category_id_counts_lag_1,shop_id_week_3_sum_lag_1,shop_id_week_4_sum_lag_1,item_id_week_3_sum_lag_1,item_id_week_4_sum_lag_1,item_category_id_week_3_sum_lag_1,item_category_id_week_4_sum_lag_1,shop_id_week_3_mean_lag_1,shop_id_week_4_mean_lag_1,item_id_week_3_mean_lag_1,item_id_week_4_mean_lag_1,item_category_id_week_3_mean_lag_1,item_category_id_week_4_mean_lag_1,shop_id_week_3_count_lag_1,shop_id_week_4_count_lag_1,item_id_week_3_count_lag_1,item_id_week_4_count_lag_1,item_category_id_week_3_count_lag_1,item_category_id_week_4_count_lag_1
0,0,2,19,0.0,0.0,1,40,4,42,0,11,4,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,42,33,2.0,499.0,1,37,4,562,19,11,1,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,42,32,11.0,349.0,1,40,4,42,19,11,4,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,42,29,0.0,0.0,1,23,123,42,19,5,16,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,42,28,0.0,0.0,1,30,107,42,19,8,55,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127999,34,21,5232,0.0,0.0,11,23,123,617,13,5,16,30,0.0,1.0,2.0,0.0,28.0,1746.0,2182.0,0.636230,0.322510,0.337402,0.0,37.0,0.0,0.251709,512.0,338.000,0.0,295.00000,30.0,1567.0,2087.0,446.0,356.0,7.0,3.0,463.0,418.0,1.259887,1.174917,1.0,0.600000,1.033482,1.019512,354.0,303.0,7.0,5.0,448.0,410.0
11128000,34,21,5268,0.0,0.0,11,20,94,42,13,5,11,30,0.0,0.0,0.0,0.0,0.0,1746.0,5676.0,0.000000,0.322510,1.141602,0.0,204.0,0.0,1.805664,0.0,1057.000,0.0,1509.00000,0.0,1567.0,4058.0,446.0,356.0,NaN,NaN,2297.0,1013.0,1.259887,1.174917,NaN,NaN,2.007867,1.211722,354.0,303.0,NaN,NaN,1144.0,836.0
11128001,34,21,5039,0.0,0.0,11,23,129,42,13,5,16,30,1.0,1.0,3.0,1499.0,29.0,1746.0,2182.0,0.659180,0.322510,0.337402,1.0,37.0,1.0,0.251709,681.5,338.000,1499.0,295.00000,28.0,1567.0,2087.0,446.0,356.0,9.0,4.0,463.0,418.0,1.259887,1.174917,1.0,1.000000,1.033482,1.019512,354.0,303.0,9.0,4.0,448.0,410.0
11128002,34,21,5672,0.0,0.0,11,3,4,655,13,1,11,30,1.0,4.0,4.0,3590.0,57.0,1746.0,992.0,1.295898,0.322510,0.751465,1.0,42.0,1.0,1.400391,2106.0,797.000,3590.0,1365.00000,55.0,1567.0,903.0,446.0,356.0,16.0,13.0,227.0,155.0,1.259887,1.174917,1.0,1.083333,1.118227,1.123188,354.0,303.0,16.0,12.0,203.0,138.0


In [ ]:
col3 = [pd.Grouper(key="date",freq="W"),"item_id","shop_id"]
col4 = [pd.Grouper(key="date",freq="W"),"item_category_id","shop_id"]

In [ ]:
for i in range(1,2):
    for f in ["sum"]:
        g_ = pd.merge(g_, generate_weekly_sales(col3,f,lag=i), how="left",on=col3[1:]+["date_block_num"] )
        g_ = pd.merge(g_, generate_weekly_sales(col4,f,lag=i), how="left",on=col4[1:]+["date_block_num"] )
    
    

weekly data sum created.
weekly data sum created.


In [ ]:
col2 = [pd.Grouper(key="date",freq="15D"),"shop_id"]
col3 = [pd.Grouper(key="date",freq="15D"),"item_id"]
col4 = [pd.Grouper(key="date",freq="15D"),"item_category_id"]


In [ ]:
for i in range(1,2):
    for f in ["sum","mean","count"]:
        g_ = pd.merge(g_, get_last_15_day_sale(sales_,col2,"item_cnt_day",f,lag=i), how="left",on=col2[1:]+["date_block_num"] )
        g_ = pd.merge(g_, get_last_15_day_sale(sales_,col3,"item_cnt_day",f,lag=i), how="left",on=col3[1:]+["date_block_num"] )
        g_ = pd.merge(g_, get_last_15_day_sale(sales_,col4,"item_cnt_day",f,lag=i), how="left",on=col4[1:]+["date_block_num"] )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week h

15 day var created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week h

15 day var created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week h

...

15 day var created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week h

15 day var created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week h

15 day var created


In [ ]:
# g_[f'week_4_sum_zero_lag_1'] = (g_.item_id_week_4_sum_lag_1 > 0)
# g_[f'week_4_mean_zero_lag_1'] = (g_.item_id_week_4_mean_lag_1 > 0)
# g_[f'week_3_sum_zero_lag_1'] = (g_.item_id_week_3_sum_lag_1 > 0)

In [ ]:
g_.fillna(0,inplace=True)

In [ ]:
del sales_
del sales
gc.collect()

414

In [ ]:
g_.fillna(0,inplace=True)

In [ ]:
generate_mean_encodings(g_,["item_id"])
generate_mean_encodings(g_,["shop_id"])
generate_mean_encodings(g_,["item_category_id"])
generate_mean_encodings(g_,["item_id"],target_col="item_price")

enc_mean_item_id_item_cnt_day created.
enc_mean_shop_id_item_cnt_day created.
enc_mean_item_category_id_item_cnt_day created.
enc_mean_item_id_item_price created.


In [ ]:
generate_mean_encodings(g_,["item_category_id","shop_id"])
generate_mean_encodings(g_,["item_id","shop_id"])

enc_mean_item_category_id_shop_id_item_cnt_day created.
enc_mean_item_id_shop_id_item_cnt_day created.


In [ ]:
g_.isna().sum()

date_block_num                                    0
shop_id                                           0
item_id                                           0
item_cnt_day                                      0
item_price                                        0
                                                 ..
enc_mean_shop_id_item_cnt_day                     0
enc_mean_item_category_id_item_cnt_day            0
enc_mean_item_id_item_price                       0
enc_mean_item_category_id_shop_id_item_cnt_day    0
enc_mean_item_id_shop_id_item_cnt_day             0
Length: 80, dtype: int64

In [ ]:
g_

,date_block_num,shop_id,item_id,item_cnt_day,item_price,month,item_category_id,name2,name3,city,type,subtype,days,lag_1,lag_2,lag_3,price_lag_1,item_sum_sale_lag_1,shop_sum_sale_lag_1,item_cat_sum_sale_lag_1,item_mean_sale_lag_1,shop_mean_sale_lag_1,item_cat_mean_sale_lag_1,item_shop_sum_sale_lag_1,item_cat_shop_sum_sale_lag_1,item_shop_mean_sale_lag_1,item_cat_shop_mean_sale_lag_1,item_mean_price_lag_1,item_category_mean_price_lag_1,item_shop_mean_price_lag_1,item_category_shop_mean_price_lag_1,item_id_counts_lag_1,shop_id_counts_lag_1,item_category_id_counts_lag_1,shop_id_week_3_sum_lag_1,shop_id_week_4_sum_lag_1,item_id_week_3_sum_lag_1,item_id_week_4_sum_lag_1,item_category_id_week_3_sum_lag_1,item_category_id_week_4_sum_lag_1,shop_id_week_3_mean_lag_1,shop_id_week_4_mean_lag_1,item_id_week_3_mean_lag_1,item_id_week_4_mean_lag_1,item_category_id_week_3_mean_lag_1,item_category_id_week_4_mean_lag_1,shop_id_week_3_count_lag_1,shop_id_week_4_count_lag_1,item_id_week_3_count_lag_1,item_id_week_4_count_lag_1,item_category_id_week_3_count_lag_1,item_category_id_week_4_count_lag_1,item_id_shop_id_week_3_sum_lag_1,item_id_shop_id_week_4_sum_lag_1,item_category_id_shop_id_week_3_sum_lag_1,item_category_id_shop_id_week_4_sum_lag_1,shop_id_15D_0_sum_lag_1,shop_id_15D_1_sum_lag_1,item_id_15D_0_sum_lag_1,item_id_15D_1_sum_lag_1,item_category_id_15D_0_sum_lag_1,item_category_id_15D_1_sum_lag_1,shop_id_15D_0_mean_lag_1,shop_id_15D_1_mean_lag_1,item_id_15D_0_mean_lag_1,item_id_15D_1_mean_lag_1,item_category_id_15D_0_mean_lag_1,item_category_id_15D_1_mean_lag_1,shop_id_15D_0_count_lag_1,shop_id_15D_1_count_lag_1,item_id_15D_0_count_lag_1,item_id_15D_1_count_lag_1,item_category_id_15D_0_count_lag_1,item_category_id_15D_1_count_lag_1,enc_mean_item_id_item_cnt_day,enc_mean_shop_id_item_cnt_day,enc_mean_item_category_id_item_cnt_day,enc_mean_item_id_item_price,enc_mean_item_category_id_shop_id_item_cnt_day,enc_mean_item_id_shop_id_item_cnt_day
0,0,2,19,0.0,0.0,1,40,4,42,0,11,4,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
1,0,42,33,2.0,499.0,1,37,4,562,19,11,1,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
2,0,42,32,11.0,349.0,1,40,4,42,19,11,4,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
3,0,42,29,0.0,0.0,1,23,123,42,19,5,16,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
4,0,42,28,0.0,0.0,1,30,107,42,19,8,55,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000

In [ ]:
g_.to_pickle("df.pkl")

In [ ]:
g_ = pd.read_pickle("df.pkl")


Memory usage of dataframe is 4818.07 MB
Memory usage after optimization is: 1698.00 MB
Decreased by 64.8%


In [ ]:
drop_cols = ["item_price","item_cnt_day","shop_id","item_id","item_category_id"] 


In [ ]:
test = pd.read_csv("test.csv")
test.loc[test.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 10, 'shop_id'] = 11

Xtr = g_[g_.date_block_num < 33].drop(drop_cols,axis=1)
Xva = g_[g_.date_block_num == 33].drop(drop_cols,axis=1)
Xte = g_[g_.date_block_num == 34]

f1 = g_[g_.date_block_num == 33].shop_id.isin(test.shop_id.unique())
f2 = g_[g_.date_block_num == 33].item_id.isin(test.item_id.unique())
yf = Xva[(f1&f2)].index

test = pd.merge(test,Xte,how="left",on=["shop_id","item_id"]).drop(drop_cols,axis=1)

del Xte

ytr = g_[g_.date_block_num < 33].item_cnt_day 
yva = g_[g_.date_block_num == 33].item_cnt_day 

del g_

In [ ]:
del items,categories,shops

In [ ]:
gc.collect()

In [ ]:
assert(Xtr.isna().values.sum() == 0)
assert(ytr.isna().values.sum() == 0)
assert(Xva.isna().values.sum() == 0)
assert(yva.isna().values.sum() == 0)
assert(test.isna().values.sum() == 0)

In [ ]:
# Xtr.is_zero_lag_1

In [ ]:
# del Xte,g_
# gc.collect()

In [ ]:
gc.collect()

In [ ]:
test.drop("ID",axis=1,inplace=True)

In [ ]:
gbm1 = lgbm.LGBMRegressor(n_estimators=100)

In [ ]:
gbm1 = gbm1.fit(Xtr[x.values],ytr,eval_set=[(Xtr[x.values],ytr),(Xva.loc[(f1&f2)][x.values],yva.loc[yf])],eval_names=["train","val"],eval_metric="rmse")

In [ ]:
# xgb1 = xgb.XGBRegressor(tree_method="gpu_hist")

In [ ]:
# xgb1.fit(Xtr,ytr,eval_set=[(Xtr,ytr),(Xva,yva)],eval_metric="rmse")

In [ ]:
ans = pd.DataFrame()
ans["ID"] = pd.read_csv("test.csv").ID
ans['item_cnt_month'] = gbm1.predict(test[x.values]).clip(0,20.0)
ans.to_csv("sub513.csv",index=None)

In [ ]:
from lightgbm import plot_importance

In [ ]:
fig,ax = plt.subplots(figsize=(15,10))
plot_importance(gbm1,ax=ax,importance_type="gain",max_num_features=40)

In [ ]:
cols_to_select = (gbm1.booster_.feature_importance(importance_type="gain")>=100000)

In [ ]:
x = pd.Series(gbm1.feature_name_)[cols_to_select]
x

In [ ]:
len(x)

In [ ]:
!unzip /content/drive/MyDrive/cgbm.zip

Archive:  /content/drive/MyDrive/cgbm.zip
  inflating: cbm.pkl                 


In [ ]:
import pickle5

In [ ]:
f = open('/content/cbm.pkl','rb')

In [ ]:
x = pickle5.load(f)

In [ ]:
pickle5.dump(x.columns.to_list(),'xgcols.pkl',protocol=4)

TypeError: ignored

In [ ]:
x.columns.tolist()

['enc_mean_item_category_id_shop_id_item_cnt_day',
 'enc_mean_item_id_item_price',
 'item_shop_sum_sale_lag_1',
 'lag_1',
 'enc_mean_item_id_item_cnt_day',
 'item_id_week_4_sum_lag_1',
 'name2',
 'enc_mean_item_id_shop_id_item_cnt_day',
 'month',
 'item_shop_mean_sale_lag_1',
 'item_id_week_3_count_lag_1',
 'item_id_week_4_count_lag_1',
 'item_id_week_3_sum_lag_1',
 'item_id_week_4_mean_lag_1',
 'enc_mean_shop_id_item_cnt_day',
 'enc_mean_item_category_id_item_cnt_day',
 'lag_2',
 'type',
 'item_cat_shop_sum_sale_lag_1',
 'lag_3',
 'item_cat_shop_mean_sale_lag_1',
 'item_id_shop_id_week_3_sum_lag_1',
 'item_sum_sale_lag_1',
 'date_block_num',
 'item_id_shop_id_week_4_sum_lag_1',
 'price_lag_1',
 'name3',
 'item_cat_sum_sale_lag_1',
 'item_id_15D_1_sum_lag_1',
 'item_id_15D_1_mean_lag_1',
 'item_id_15D_0_count_lag_1',
 'item_category_id_week_4_sum_lag_1',
 'item_id_15D_1_count_lag_1',
 'item_shop_mean_price_lag_1',
 'item_category_id_shop_id_week_3_sum_lag_1',
 'item_cat_mean_sale_lag_1